In [1]:
import os
os.chdir('../../')

In [2]:
!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Fri May  2 16:16:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.04             Driver Version: 570.124.04     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:02:00.0 Off |                  Off |
| 30%   36C    P8             12W /  450W |      87MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### from "sample pair.ipynb"

In [3]:
!ls samples/64x64_diffusion/dpmsolver++_order1_200/image_samples/images

0.npz	 113.npz  127.npz  25.npz  39.npz  52.npz  66.npz  7.npz   93.npz
100.npz  114.npz  12.npz   26.npz  3.npz   53.npz  67.npz  80.npz  94.npz
101.npz  115.npz  13.npz   27.npz  40.npz  54.npz  68.npz  81.npz  95.npz
102.npz  116.npz  14.npz   28.npz  41.npz  55.npz  69.npz  82.npz  96.npz
103.npz  117.npz  15.npz   29.npz  42.npz  56.npz  6.npz   83.npz  97.npz
104.npz  118.npz  16.npz   2.npz   43.npz  57.npz  70.npz  84.npz  98.npz
105.npz  119.npz  17.npz   30.npz  44.npz  58.npz  71.npz  85.npz  99.npz
106.npz  11.npz   18.npz   31.npz  45.npz  59.npz  72.npz  86.npz  9.npz
107.npz  120.npz  19.npz   32.npz  46.npz  5.npz   73.npz  87.npz
108.npz  121.npz  1.npz    33.npz  47.npz  60.npz  74.npz  88.npz
109.npz  122.npz  20.npz   34.npz  48.npz  61.npz  75.npz  89.npz
10.npz	 123.npz  21.npz   35.npz  49.npz  62.npz  76.npz  8.npz
110.npz  124.npz  22.npz   36.npz  4.npz   63.npz  77.npz  90.npz
111.npz  125.npz  23.npz   37.npz  50.npz  64.npz  78.npz  91.npz
112.npz  126.npz 

In [4]:
import numpy as np
import torch
import torch.nn.functional as F

N = 128
M = 128
K = 1
root_dir = 'samples/64x64_diffusion/dpmsolver++_order1_200/image_samples/images'
npz_files = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if '.npz' in f]

trajs = []
timesteps = []
for file in npz_files[:16]:
    data = np.load(file)
    trajs.append(data['traj'])
    timesteps.append(data['timesteps'])
trajs = torch.tensor(trajs).transpose(0, 1)
timesteps = timesteps[0]
print(trajs.shape, timesteps.shape)

torch.Size([201, 16, 3, 64, 64]) (201,)


/tmp/ipykernel_616609/2044542655.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  trajs = torch.tensor(trajs).transpose(0, 1)


In [5]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import random

from main import parse_args_and_config, Diffusion
from datasets import inverse_data_transform

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.benchmark = False


2025-05-02 16:16:05.860564: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 16:16:05.880833: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-02 16:16:05.880858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-02 16:16:05.881775: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-02 16:16:05.886379: I tensorflow/core/platform/cpu_feature_guar

In [6]:
!mkdir -p /data/guided-diffusion/dc/imagenet64/

In [11]:
for order in [3]:
    for NFE in [5, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40]:
        ###############################################################################
        # 1) Notebook에서 sys.argv를 직접 설정 (argparse 흉내)
        ###############################################################################
        sys.argv = [
            "main.py",
            "--config", "imagenet64.yml",  # 사용하려는 config
            "--sample",
            "--fid",
            "--dpm_solver_type", "data_prediction",
            "--dpm_solver_order", f"{order}",
            "--skip_type", "logSNR",
            "--ni",

            "--sample_type", "dcsolver",
            "--timesteps", f"{NFE}",
            "--dc_dir", "/data/guided-diffusion/dc/imagenet64/",
        ]

        ###############################################################################
        # 2) 인자/설정 로드
        ###############################################################################
        args, config = parse_args_and_config()

        ###############################################################################
        # 3) Diffusion 객체 생성 -> 모델 로딩
        ###############################################################################
        diffusion = Diffusion(args, config, rank=0)
        diffusion.prepare_model()
        diffusion.model.eval()

        ###############################################################################
        # 4) 배치(25장) 한 번에 샘플링 -> 5x5 그리드(여백 없이) 시각화
        ###############################################################################
        device = diffusion.device
        trajs = trajs.to(device)
        pred, _ = diffusion.sample_image(trajs[0], diffusion.model, target=(trajs, timesteps))
        print('NFE :', NFE, 'order :', order, 'loss :', F.mse_loss(trajs[-1], pred))


INFO - main.py - 2025-05-02 16:17:21,828 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:21,828 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:21,828 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:21,828 - Using device: cuda


[prepare_model] Model is ready.


INFO - main.py - 2025-05-02 16:17:31,148 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:31,148 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:31,148 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:31,148 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:31,148 - Using device: cuda


/data/guided-diffusion/dc/imagenet64/NFE=5,p=3.npz  saved!
dcsolver
NFE : 5 order : 3 loss : tensor(0.0583, device='cuda:0')
[prepare_model] Model is ready.


INFO - main.py - 2025-05-02 16:17:44,784 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:44,784 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:44,784 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:44,784 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:44,784 - Using device: cuda
INFO - main.py - 2025-05-02 16:17:44,784 - Using device: cuda


...

/data/guided-diffusion/dc/imagenet64/NFE=25,p=3.npz  saved!
dcsolver
NFE : 25 order : 3 loss : tensor(0.0001, device='cuda:0')
[prepare_model] Model is ready.


INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda
INFO - main.py - 2025-05-02 16:25:00,135 - Using device: cuda


/data/guided-diffusion/dc/imagenet64/NFE=30,p=3.npz  saved!
dcsolver
NFE : 30 order : 3 loss : tensor(0.0002, device='cuda:0')
[prepare_model] Model is ready.


INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda
INFO - main.py - 2025-05-02 16:27:19,825 - Using device: cuda


/data/guided-diffusion/dc/imagenet64/NFE=35,p=3.npz  saved!
dcsolver
NFE : 35 order : 3 loss : tensor(0.0003, device='cuda:0')
[prepare_model] Model is ready.
/data/guided-diffusion/dc/imagenet64/NFE=40,p=3.npz  saved!
dcsolver
NFE : 40 order : 3 loss : tensor(0.0002, device='cuda:0')


In [15]:
data = np.load('/data/guided-diffusion/dc/imagenet64/NFE=40,p=3.npz')
data['dc_ratios']

array([ 1.        ,  1.        ,  0.73951977,  0.79444087,  0.86400801,
        0.89810824,  0.79757202,  0.87910068,  0.86734504,  0.85085368,
        0.86095625,  0.85885626, -0.77172303,  1.01838243,  1.34770083,
        0.44733763,  0.83823639,  0.91461462,  1.30667841,  0.43183535,
        0.92643923,  0.92441142,  0.81046242,  0.8445676 ,  0.85318923,
        0.84935844,  0.85597777,  0.8632955 ,  0.86320835,  0.80640626,
        0.8510704 ,  0.82124048,  0.84072918,  0.82903969,  0.86127883,
        0.83582997,  0.90094668,  0.87084967,  0.87426615,  1.37519038])

In [16]:
!ls /data/guided-diffusion/dc/imagenet64/

'NFE=10,p=3.npz'  'NFE=20,p=3.npz'  'NFE=35,p=3.npz'  'NFE=6,p=3.npz'
'NFE=12,p=3.npz'  'NFE=25,p=3.npz'  'NFE=40,p=3.npz'  'NFE=8,p=3.npz'
'NFE=15,p=3.npz'  'NFE=30,p=3.npz'  'NFE=5,p=3.npz'
